In [1]:
class AVLTreeNode:
    """Represents a node in an AVL Tree."""
    def __init__(self, value):
        """Initialize the node with a value, and set its height to 1."""
        self.value = value
        self.left_child = None
        self.right_child = None
        self.height = 1  # New node is initially a leaf

class AVLTree:
    """Represents an AVL Tree."""
    def __init__(self):
        """Initialize an empty AVL Tree with no root."""
        self.root_node = None  # Root of the AVL Tree

    def _get_height(self, node):
        """Return the height of the node, or 0 if the node is None."""
        if not node:
            return 0
        return node.height

    def _get_balance_factor(self, node):
        """Calculate and return the balance factor of the node."""
        if not node:
            return 0
        return self._get_height(node.left_child) - self._get_height(node.right_child)

    def _right_rotate(self, y):
        """Perform a right rotation around the given node."""
        x = y.left_child
        T2 = x.right_child

        # Rotate
        x.right_child = y
        y.left_child = T2

        # Update heights
        y.height = max(self._get_height(y.left_child), self._get_height(y.right_child)) + 1
        x.height = max(self._get_height(x.left_child), self._get_height(x.right_child)) + 1

        # Return new root
        return x

    def _left_rotate(self, x):
        """Perform a left rotation around the given node."""
        y = x.right_child
        T2 = y.left_child

        # Rotate
        y.left_child = x
        x.right_child = T2

        # Update heights
        x.height = max(self._get_height(x.left_child), self._get_height(x.right_child)) + 1
        y.height = max(self._get_height(y.left_child), self._get_height(y.right_child)) + 1

        # Return new root
        return y

    def _insert_node(self, current_node, value):
        """Insert a value into the AVL Tree, maintaining AVL properties."""
        if not current_node:
            return AVLTreeNode(value)
        elif value < current_node.value:
            current_node.left_child = self._insert_node(current_node.left_child, value)
        else:
            current_node.right_child = self._insert_node(current_node.right_child, value)

        # Update height of the ancestor node
        current_node.height = 1 + max(self._get_height(current_node.left_child), self._get_height(current_node.right_child))

        # Get the balance factor to check if the node became unbalanced
        balance = self._get_balance_factor(current_node)

        # Perform rotations to balance the tree

        # Left Left Case
        if balance > 1 and value < current_node.left_child.value:
            return self._right_rotate(current_node)

        # Right Right Case
        if balance < -1 and value > current_node.right_child.value:
            return self._left_rotate(current_node)

        # Left Right Case
        if balance > 1 and value > current_node.left_child.value:
            current_node.left_child = self._left_rotate(current_node.left_child)
            return self._right_rotate(current_node)

        # Right Left Case
        if balance < -1 and value < current_node.right_child.value:
            current_node.right_child = self._right_rotate(current_node.right_child)
            return self._left_rotate(current_node)

        # Return the unchanged node pointer
        return current_node

    def insert(self, value):
        """Insert a value into the AVL Tree."""
        self.root_node = self._insert_node(self.root_node, value)

    def _find_min_value_node(self, node):
        """Return the node with the minimum value in the subtree."""
        if node is None or node.left_child is None:
            return node
        return self._find_min_value_node(node.left_child)

    def _delete_node(self, current_node, value):
        """Delete a value from the AVL Tree, maintaining AVL properties."""
        if not current_node:
            return current_node

        if value < current_node.value:
            current_node.left_child = self._delete_node(current_node.left_child, value)
        elif value > current_node.value:
            current_node.right_child = self._delete_node(current_node.right_child, value)
        else:
            # Node with one or no child
            if current_node.left_child is None:
                return current_node.right_child
            elif current_node.right_child is None:
                return current_node.left_child

            # Node with two children: Get the inorder successor
            temp_node = self._find_min_value_node(current_node.right_child)

            # Copy the inorder successor's content to this node
            current_node.value = temp_node.value

            # Delete the inorder successor
            current_node.right_child = self._delete_node(current_node.right_child, temp_node.value)

        # If the tree has only one node, return it
        if current_node is None:
            return current_node

        # Update height of the current node
        current_node.height = 1 + max(self._get_height(current_node.left_child), self._get_height(current_node.right_child))

        # Get the balance factor
        balance = self._get_balance_factor(current_node)

        # Perform rotations to balance the tree

        # Left Left Case
        if balance > 1 and self._get_balance_factor(current_node.left_child) >= 0:
            return self._right_rotate(current_node)

        # Left Right Case
        if balance > 1 and self._get_balance_factor(current_node.left_child) < 0:
            current_node.left_child = self._left_rotate(current_node.left_child)
            return self._right_rotate(current_node)

        # Right Right Case
        if balance < -1 and self._get_balance_factor(current_node.right_child) <= 0:
            return self._left_rotate(current_node)

        # Right Left Case
        if balance < -1 and self._get_balance_factor(current_node.right_child) > 0:
            current_node.right_child = self._right_rotate(current_node.right_child)
            return self._left_rotate(current_node)

        return current_node

    def delete(self, value):
        """Delete a value from the AVL Tree."""
        self.root_node = self._delete_node(self.root_node, value)

    def search(self, current_node, value):
        """Search for a value in the AVL Tree."""
        if current_node is None or current_node.value == value:
            return current_node
        if value < current_node.value:
            return self.search(current_node.left_child, value)
        return self.search(current_node.right_child, value)

    def inorder_traversal(self, current_node):
        """Return the in-order traversal of the tree as a list of values."""
        return self._inorder_helper(current_node, [])

    def _inorder_helper(self, node, result):
        """Helper method for in-order traversal."""
        if node is not None:
            self._inorder_helper(node.left_child, result)
            result.append(node.value)
            self._inorder_helper(node.right_child, result)
        return result

# Test the AVL Tree
if __name__ == "__main__":
    avl_tree = AVLTree()

    # Insert some nodes
    avl_tree.insert(10)
    avl_tree.insert(20)
    avl_tree.insert(30)
    avl_tree.insert(40)
    avl_tree.insert(50)
    avl_tree.insert(25)

    # Print in-order traversal of the AVL tree (should be sorted)
    print("In-order traversal:", avl_tree.inorder_traversal(avl_tree.root_node))

    # Search for nodes
    print("Search for 30:", "Found" if avl_tree.search(avl_tree.root_node, 30) else "Not found")
    print("Search for 100:", "Found" if avl_tree.search(avl_tree.root_node, 100) else "Not found")

    # Delete nodes
    avl_tree.delete(50)
    print("In-order traversal after deleting 50:", avl_tree.inorder_traversal(avl_tree.root_node))

    avl_tree.delete(30)
    print("In-order traversal after deleting 30:", avl_tree.inorder_traversal(avl_tree.root_node))


In-order traversal: [10, 20, 25, 30, 40, 50]
Search for 30: Found
Search for 100: Not found
In-order traversal after deleting 50: [10, 20, 25, 30, 40]
In-order traversal after deleting 30: [10, 20, 25, 40]
